In [1]:
# importing needed libraries

import pandas as pd
import re

### 1. Making `.log` files a nicely looking dataframes.

***Getting logs into list at first.***

In [2]:
exec_logs = []
f1 = open('exec_logs.log')

for element in f1:
    exec_logs.append(element)

In [3]:
market_logs = []
f2 = open('market_logs.log')

for element in f2:
    market_logs.append(element)

In [4]:
# checking if lens are ok

print(f'Length of exec_logs: {len(exec_logs)}')
print(f'Length of market_logs: {len(market_logs)}')

Length of exec_logs: 8682
Length of market_logs: 397802


#### 1.1. Preprocessing exec_logs.

In [5]:
# starting creating a dataframe

data_exec = pd.DataFrame([element.split(', ') for element in exec_logs])
data_exec.columns = ['traded_at', 'side', 'price', 'traded_volume', 'volume_left', 'delta_execsend']

In [6]:
data_exec.head()

,traded_at,side,price,traded_volume,volume_left,delta_execsend
0,eLog: order exec at 1622505601222248272 with i...,side 1,price 1108.6,traded volume 1,volume left on level 10,delta_execsend 30451968\n
1,eLog: order exec at 1622505601424113404 with i...,side -1,price 1108.4,traded volume 1,volume left on level 22,delta_execsend 17118720\n
2,eLog: order exec at 1622505601604667896 with i...,side 1,price 1108.3,traded volume 1,volume left on level 2,delta_execsend 153866496\n
3,eLog: order exec at 1622505602982200992 with i...,side -1,price 1108.4,traded volume 1,volume left on level 1,delta_execsend 153371136\n
4,eLog: order exec at 1622505603600579508 with i...,side 1,price 1108.3,traded volume 1,volume left on level 6,delta_execsend 618378496\n


In [7]:
# getting rid of unneeded words such as side, price, etc.

data_exec['traded_at'] = data_exec['traded_at'].map(lambda x: str(x)[20:39])
data_exec['side'] = data_exec['side'].map(lambda x: str(x)[4:])
data_exec['price'] = data_exec['price'].map(lambda x: str(x)[6:])
data_exec['traded_volume'] = data_exec['traded_volume'].map(lambda x: str(x)[14:])
data_exec['volume_left'] = data_exec['volume_left'].map(lambda x: str(x)[21:])
data_exec['delta_execsend'] = data_exec['delta_execsend'].map(lambda x: str(x)[15:-2])

In [8]:
# modifying last value cause it didn't have '\n' but still lost 2 digits.

data_exec['delta_execsend'][8681] = '14734981632'

In [9]:
# changing the types

data_exec['traded_at'] = (data_exec['traded_at'].astype('int64') / 1_000_000_000).astype('int64')
data_exec['side'] = data_exec['side'].astype('int64')
data_exec['price'] = data_exec['price'].astype(float)
data_exec['traded_volume'] = data_exec['traded_volume'].astype('int64')
data_exec['volume_left'] = data_exec['volume_left'].astype('int64')
data_exec['delta_execsend'] = data_exec['delta_execsend'].astype('int64')

In [10]:
# looks cool now!

data_exec.sample(5)

,traded_at,side,price,traded_volume,volume_left,delta_execsend
5776,1626316477,1,1146.7,1,7,1445915878
2452,1624406520,-1,1133.0,1,1,611869260
7549,1627350215,1,1150.1,1,577,200345
83,1622509119,-1,1108.3,1,167,8127278899
3166,1624844466,1,1129.8,1,630,6362885888


### 1.2. Preprocessing market_logs.

In [11]:
# starting creating a dataframe!

data_market = pd.DataFrame([element.split(' ') for element in market_logs])
data_market = data_market.drop(columns = [0,1,2,3,5,6,8,10,11,13,14,15])
data_market.columns = ['traded_at', 'side', 'price', 'traded_volume', 'dom']

In [12]:
data_market.head()

,traded_at,side,price,traded_volume,dom
0,1622505601191796236,"-1,","1108.9,",36,6@1108.7x1108.8@104\n
1,1622505601222248272,"-1,","1108.7,",17,10@1108.5x1108.6@23\n
2,1622505601364445668,"-1,","1108.5,",20,16@1108.3x1108.6@23\n
3,1622505601405325968,"1,","1108.6,",1,16@1108.3x1108.6@22\n
4,1622505601406994624,"-1,","1108.3,",16,18@1108.2x1108.3@12\n


In [13]:
# getting rid of commas at the end

data_market['side'] = data_market['side'].map(lambda x: str(x)[:-1])
data_market['price'] = data_market['price'].map(lambda x: str(x)[:-1])
data_market['dom'] = data_market['dom'].map(lambda x: str(x)[:-2])

Now I want to split **`dom`** column into 4 different variables.

In [14]:
# to get volume_bid
def volume_bid(row):
        return re.findall(r'^\w+', row)[0]

# to get volume_ask
def volume_ask(row):
    try:
        return re.findall(r'\w+$', row)[0]
    except:
        return 0
    
# to get bid_price
def bid_price(row):
    return re.split(r'x', row)[0].split('@')[1]

# to get ask_price
def ask_price(row):
    return re.split(r'x', row)[1].split('@')[0]

In [15]:
# applying functions and creating new columns!

data_market['volume_bid'] = data_market['dom'].apply(volume_bid)
data_market['volume_ask'] = data_market['dom'].apply(volume_ask)
data_market['bid_price'] = data_market['dom'].apply(bid_price)
data_market['ask_price'] = data_market['dom'].apply(ask_price)

In [16]:
# changing data types to proper ones

data_market['traded_at'] = data_market['traded_at'].astype('int64')
data_market['side'] = data_market['side'].astype('int64')
data_market['price'] = data_market['price'].astype(float)
data_market['traded_volume'] = data_market['traded_volume'].astype('int64')

data_market['volume_bid'] = data_market['volume_bid'].astype('int64')
data_market['volume_ask'] = data_market['volume_ask'].astype('int64')
data_market['bid_price'] = data_market['bid_price'].astype(float)
data_market['ask_price'] = data_market['ask_price'].astype(float)

*i want to get unix-like time to normal time, let's do it.*

In [17]:
data_market.sample()

,traded_at,side,price,traded_volume,dom,volume_bid,volume_ask,bid_price,ask_price
304162,1626746623436286476,-1,1149.5,5,203@1149.5x1149.6@13,203,13,1149.5,1149.6


*i assume that `traded_at` column is in nanoseconds, so I want to convert it to normal seconds.*

In [18]:
data_market['traded_at'] = (data_market['traded_at'] / 1_000_000_000).astype(int)

In [19]:
# looks cool now!

data_market.head(5)

,traded_at,side,price,traded_volume,dom,volume_bid,volume_ask,bid_price,ask_price
0,1622505601,-1,1108.9,36,6@1108.7x1108.8@10,6,10,1108.7,1108.8
1,1622505601,-1,1108.7,17,10@1108.5x1108.6@2,10,2,1108.5,1108.6
2,1622505601,-1,1108.5,20,16@1108.3x1108.6@2,16,2,1108.3,1108.6
3,1622505601,1,1108.6,1,16@1108.3x1108.6@2,16,2,1108.3,1108.6
4,1622505601,-1,1108.3,16,18@1108.2x1108.3@1,18,1,1108.2,1108.3


### 2. Calculations

*1. We need to calculate mid_price between ask_price and bid_price, let's do it!*

In [20]:
data_market['mid_price'] = (data_market['ask_price'] + data_market['bid_price']) / 2

*2. We need to calculate dmid5 which is calculated by the following formula:*

***dmid5*** = (mid_price in 5 seconds **-** price of data_exec) * side

*i'll create 2 dataframes to get only values i want to work with.*

In [21]:
final_market = data_market.drop(columns = ['price', 'traded_volume', 'side',
                                           'dom', 'volume_bid', 'volume_ask',
                                           'bid_price', 'ask_price'])

In [22]:
final_exec = data_exec.drop(columns = ['traded_volume', 'volume_left'])

*let's create a target column to merge correctly.*

In [23]:
# +5 cause we wanna get mid_price in 5 seconds

final_exec['traded_at'] = final_exec['traded_at'] + 5

In [24]:
display(final_market.head())
display(final_exec.head())

,traded_at,mid_price
0,1622505601,1108.75
1,1622505601,1108.55
2,1622505601,1108.45
3,1622505601,1108.45
4,1622505601,1108.25


,traded_at,side,price,delta_execsend
0,1622505606,1,1108.6,3045196
1,1622505606,-1,1108.4,1711872
2,1622505606,1,1108.3,15386649
3,1622505607,-1,1108.4,15337113
4,1622505608,1,1108.3,61837849


In [25]:
final_table = final_exec.merge(final_market, how = 'inner')

In [26]:
final_table

,traded_at,side,price,delta_execsend,mid_price
0,1622505606,1,1108.6,3045196,1108.65
1,1622505606,1,1108.6,3045196,1108.65
2,1622505606,1,1108.6,3045196,1108.45
3,1622505606,1,1108.6,3045196,1108.45
4,1622505606,1,1108.6,3045196,1108.50
...,...,...,...,...,...
5827,1627625779,1,1149.2,1918210918,1149.15
5828,1627626048,-1,1149.5,2515204659,1149.60
5829,1627626478,-1,1150.4,1213418470,1150.45
5830,1627626478,-1,1150.4,1213418470,1150.45


In [27]:
final_table['dmid5'] = (final_table['mid_price'] - final_table['price']) * final_table['side']
final_table['delta_execsend'] = final_table['delta_execsend'] / 1_000_000_000

In [28]:
print('Correlation between dmid5 and delta_execsend is:')
print(final_table['dmid5'].corr(final_table['delta_execsend']))

Correlation between dmid5 and delta_execsend is:
0.011734153655517322


#### The correlation between dmid5 and delta_execsend is almost 0, to be accurate ~0.01.

(hopefully I did everything right)